# Stage 1 - Data Preparation and Research Question Defining

## Data Preparation

### Data Acquisition and cleaning

In [ ]:
import pandas as pd
from pathlib import Path

try:
    df = pd.read_csv("all_stocks.csv")
# Create combined dataframe and write to CSV if not found
except FileNotFoundError:
    df = pd.DataFrame()
    data_path = Path("./sp500/")
    stock_files = sorted(data_path.glob("*.csv"))
    
    
    for file in stock_files:
        df_stock = pd.read_csv(file).assign(stock=file.stem)
        df = pd.concat([df, df_stock], ignore_index=True)

    # We choose records from 2018-2022
    df["Date"] = pd.to_datetime(df["Date"], dayfirst = True, format="%d-%m-%Y")
    df = df[(df["Date"] >= "2018-01-01") & (df["Date"] <= "2022-12-31")]
    df.to_csv("all_stocks.csv", index=False)

In [ ]:
df.head()

print(df.info())

In [ ]:
# set stock column as categorical data
df["stock"] = df["stock"].astype("category")

# Check for missing values by stock
def summary_na(df: pd.DataFrame) -> pd.DataFrame:
    summary = (
        df.assign(any_na=df.isna().any(axis=1))
        .groupby("stock", observed=True)
        .agg(rows=("any_na", "size"),
            rows_any_na=("any_na", "sum"))
        .assign(rate_any_na=lambda d: d["rows_any_na"] / d["rows"])
        .sort_values("rate_any_na", ascending=False)
    )
    return summary

summary = summary_na(df)
print(summary[summary["rate_any_na"] != 0])

The rate of missing value in `CTQ`, `BHI`, `SONC`, `CPICQ` is too high, we decide to drop these stocks, for the rest of stocks, we might apply interpolate in the future

In [ ]:
df = df[~df["stock"].isin(["CTQ", "BHI", "SONC", "CPICQ"])].copy()

summary_na_dropped = summary_na(df)
print(summary_na_dropped[summary_na_dropped["rate_any_na"] != 0])

In [ ]:
# Check rest Missing Values in rows
na_rows = df[df.isna().any(axis=1)].copy()
na_rows

The missing values are located in the last few months of stock data. We simply drop them.

In [ ]:
df = df.dropna()
df[df.isna().any(axis=1)]

In [ ]:
df['Volume'] = df['Volume'].astype(int)

In [ ]:
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].round(2)

In [ ]:
df.head()

In [ ]:
df.to_csv("all_stocks_cleaned.csv", index=False)

### Compute features for trading strategy

In [ ]:
cols = ["MA_5","MA_10","MA_20","MA_60","MA_120","MA_180", "MA_signal", "MA_Weight", "MA_Position", "Upper_Band_20", "Lower_Band_20", "Upper_Band_60", "Lower_Band_60", "BB_signal", "BB_Position", "BB_Weight"]
df_samples = pd.DataFrame(columns=df.columns.tolist() + cols)
sid = 0

df_grouped = df.groupby('stock')
for name, group in df_grouped:
    # Calculate moving averages for 5, 10, 20, 60, 120, 180
    group["MA_5"] = group["Close"].rolling(window=5).mean()
    group["MA_10"] = group["Close"].rolling(window=10).mean()
    group["MA_20"] = group["Close"].rolling(window=20).mean()
    group["MA_60"] = group["Close"].rolling(window=60).mean()
    group["MA_120"] = group["Close"].rolling(window=120).mean()
    group["MA_180"] = group["Close"].rolling(window=180).mean()
    group["MA_signal"] = 0
    group["MA_Weight"] = 0
    group["MA_Position"] = 0

    # Bollinger (window=20, k=2, ddof=0) (window=60, k=2.5, ddof=0)
    sd20 = group["Close"].rolling(20, min_periods=20).std(ddof=0)
    sd60 = group["Close"].rolling(60, min_periods=60).std(ddof=0)
    group["Upper_Band_20"] = group["MA_20"] + 2.0 * sd20
    group["Lower_Band_20"] = group["MA_20"] - 2.0 * sd20
    group["Upper_Band_60"] = group["MA_60"] + 2.5 * sd60
    group["Lower_Band_60"] = group["MA_60"] - 2.5 * sd60
    group["BB_signal"] = 0
    group["BB_Position"] = 0
    group["BB_Weight"] = 0

    df.loc[group.index, cols] = group[cols]

    Window_size = 180
    Pace = 90
    Padding = 180
    
    n = len(group)
    for i in range(Padding, n - Window_size + 1, Pace):
        sub = group.iloc[i:i+Window_size][df.columns.tolist()].copy()
        sub["sample_id"] = sid
        sid += 1
        df_samples = pd.concat([df_samples, sub], ignore_index=True)

In [ ]:
df_samples = df_samples[["sample_id"] + [c for c in df_samples.columns if c != "sample_id"]]
df_samples["sample_id"] = df_samples["sample_id"].astype(int)
df_samples.to_csv("time_period_samples.csv", index=False)

In [ ]:
# Trading Strategy Backtesting
from TradingStrategies import ma_strategy, backtest_ma_strategy, bb_strategy, backtest_bb_strategy
# MA Crossover Strategy
results = pd.DataFrame(columns=['sample_id', 'MA_sharpe', 'MA_cagr', 'MA_mdd', 'BB_sharpe', 'BB_cagr', 'BB_mdd'])
ma_pairs = [(5, 20), (10, 60), (20, 120), (60, 180)]

for name, group in df_samples.groupby('sample_id'):
    group = ma_strategy(group, ma_signals=ma_pairs)
    MA_sharpe, MA_cagr, MA_mdd = backtest_ma_strategy(group)

    group = bb_strategy(group)
    BB_sharpe, BB_cagr, BB_mdd = backtest_bb_strategy(group)
    results = pd.concat([results, pd.DataFrame({'sample_id': [name], 'MA_sharpe': [MA_sharpe], 'MA_cagr': [MA_cagr], 'MA_mdd': [MA_mdd], 'BB_sharpe': [BB_sharpe], 'BB_cagr': [BB_cagr], 'BB_mdd': [BB_mdd]})], ignore_index=True)


In [ ]:
prop = (results['MA_sharpe'] > 1).mean()
prop2 = (results['MA_cagr'] > 0.05).mean()
print(prop)
print(prop2)

In [ ]:
prop = (results['BB_sharpe'] > 1).mean()
prop2 = (results['BB_cagr'] > 0.05).mean()
print(prop)
print(prop2)

In [ ]:
prop = ((results['BB_sharpe'] > 1) & (results['MA_sharpe'] < 1)).mean() / (results['BB_sharpe'] > 1).mean()

print(prop)
